In [19]:
%pip install wandb -q
import wandb
from wandb.keras import WandbCallback

In [20]:
# Essentials
import numpy as np
from matplotlib import pyplot
import tensorflow
from tensorflow import keras
from keras import regularizers
from keras.models import Sequential
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.applications import InceptionV3, ResNet50, InceptionResNetV2, Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, MaxPooling2D, Activation
from keras.callbacks import  Callback, EarlyStopping

import random
import imageio
import os
import cv2
import glob
random.seed(42)

In [21]:
# Fetch the dataset form Github
!git clone https://github.com/borate267/inaturalist-dataset.git

fatal: destination path 'inaturalist-dataset' already exists and is not an empty directory.


In [22]:
# Define the labels for the Simpsons characters we're detecting
class_names = {0:'Amphibia', 1:'Animalia', 2:'Arachnida',3: 'Aves',4: 'Fungi',
              5: 'Insecta', 6:'Mammalia', 7:'Mollusca', 8:'Plantae',9: 'Reptilia'}
num_classes = 10
img_size = 224
dir1 = 'inaturalist-dataset/train'
dir2 = 'inaturalist-dataset/val'

import random

# Load training data
X_train = []
y_train = []
X_test = []
y_test = []

for label, name in class_names.items():

   list_images1 = os.listdir(dir1+'/'+name)
   list_images2 = os.listdir(dir2+'/'+name)

   for image_name in list_images1:
       image = imageio.imread(dir1+'/'+name+'/'+image_name)
       if np.ndim(image) == 3:
          X_train.append(cv2.resize(image, (img_size,img_size)))
          y_train.append(label)
   
   for image_name in list_images2:
       image = imageio.imread(dir2+'/'+name+'/'+image_name)
       if np.ndim(image) == 3:
          X_test.append(cv2.resize(image, (img_size,img_size)))
          y_test.append(label)


In [23]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# Normalize the data
X_train = X_train/255.0
X_test = X_test/255.0

# One hot encode the labels 
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [24]:
sweep_config = {
    'method': 'bayes', 
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        
        'freeze_before' : {
            'values': ['50','70','100']
        },
       
       # 'models' :{
        #    'values' : ['IV3','IRNV2', 'RN50', 'XCP']
       # },
        'dropout': {
            'values': [0, 0.2, 0.4]
        },     
        'batch_size': {
            'values': [32, 64]
        },
        'num_dense':{
            'values': [64, 128, 256, 512]
        }
    }
}

In [25]:
# Initialize a new sweep

sweep_id = wandb.sweep(sweep_config, entity="bharatik", project="cs6910assignment2")

Create sweep with ID: ywzvoorq
Sweep URL: https://wandb.ai/bharatik/cs6910assignment2/sweeps/ywzvoorq


In [26]:
input_shape = (224,224,3)

# Comment the next line if you want to sweep over different models
base_model =  InceptionResNetV2(input_shape = input_shape, include_top = False, weights = 'imagenet')

def train():
    
    config_defaults = {
        'freeze_before' : 50,
        #'models' : 'IV3',
        'dropout': 0.2,
        'batch_size': 64,
        'num_dense': 256,
        'num_classes': 10,
        'epochs' : 5
    }

     # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    config = wandb.config
    wandb.run.name = 'model_IncResV2_num_dense_'+ str(config.num_dense)+'_bs_'+str(config.batch_size)

    

    # Defining models
  #  if config.models == 'IV3':
   #     base_model =  InceptionV3(input_shape = input_shape, include_top = False, weights = 'imagenet')
   # elif config.models == 'IRNV2':
    #    base_model =  InceptionResNetV2(input_shape = input_shape, include_top = False, weights = 'imagenet')
    #elif config.models == 'RN50':
    #    base_model =  ResNet50(input_shape = input_shape, include_top = False, weights = 'imagenet')
    #elif config.models == 'XCP':
     #   base_model =  Xception(input_shape = input_shape, include_top = False, weights = 'imagenet')


    base_model.trainable = False

    stack_model = Sequential()
    stack_model.add(base_model)
    stack_model.add(Flatten())
    stack_model.add(Dense(config.num_dense, activation='relu'))
    stack_model.add(Dense(config.num_classes, activation='softmax'))
    
    final_model = stack_model
    final_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

    
    #model.fit( datagen.flow(X_train, y_train, batch_size = config.batch_size), steps_per_epoch=len(X_train)/32, epochs=config.epochs,
                       # validation_data=(X_val, y_val), callbacks = [WandbCallback()] )
    
    final_model.fit(
        x = X_train,
        y = y_train,
        batch_size = config.batch_size,
        epochs = config.epochs,
        verbose = 1,
        validation_data= (X_test, y_test),
        callbacks = [WandbCallback(),keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
    )

    ############  Fine tuning the model

    base_model.trainable = True

    # Freeze layers
    freeze_point = len(base_model.layers) - config.freeze_before
    for layer in base_model.layers[:freeze_point]:
        layer.trainable = False
    
    final_model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

    final_model.fit(
        x = X_train,
        y = y_train,
        batch_size = config.batch_size,
        epochs = 7,
        verbose = 1,
        validation_data= (X_test, y_test),
        callbacks = [WandbCallback(),keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
    )


In [ ]:
wandb.agent('ixzbgw3r',train, count = 1)